In [6]:
# def create_crack_fronts(fname):
#     crack_fronts = []
#     node_ids = []
#     with open(fname, 'r') as f:
#         for line in f:
#             if 'Crack' in line:
#                 crack_fronts.append(list(node_ids))
#                 node_ids = []
#             elif 'Number' not in line:
#                 node_ids.append(int(line))
#         if (len(node_ids) != 0):
#             crack_fronts.append(list(node_ids))
#     return crack_fronts;

# import numpy as np
# import sqlite3

# crack_fronts = create_crack_fronts('../../big-data/crack-front.txt')
# len_1 = len(crack_fronts[1])
# len_2 = len(crack_fronts[2])
# len_3 = len(crack_fronts[3])
# len_4 = len(crack_fronts[4])
# len_5 = len(crack_fronts[5])
# len_6 = len(crack_fronts[6])
# len_7 = len(crack_fronts[7])
# len_8 = len(crack_fronts[8])
# len_9 = len(crack_fronts[9])

# total = 0
# for i in range(1,10):
#     total += len(crack_fronts[i])

# print 'Total number of points: ' + str(total)
# print 'Number of points on crack front 1: ' + str(len_1)
# print 'Number of points on crack front 2: ' + str(len_2)
# print 'Number of points on crack front 3: ' + str(len_3)
# print 'Number of points on crack front 4: ' + str(len_4)
# print 'Number of points on crack front 5: ' + str(len_5)
# print 'Number of points on crack front 6: ' + str(len_6)
# print 'Number of points on crack front 7: ' + str(len_7)
# print 'Number of points on crack front 8: ' + str(len_8)
# print 'Number of points on crack front 9: ' + str(len_9)

In [7]:
# def create_tree(fname):
#     from scipy import spatial
#     x_s = []
#     y_s = []
#     z_s = []
#     ids = []

#     with open(fname, 'r') as f:
#         for line in f:
#             tokens = line.split(',')
#             x_s.append(float(tokens[0]))
#             y_s.append(float(tokens[1]))
#             z_s.append(float(tokens[2]))
#             ids.append(int(tokens[3].rstrip()))

#     tree = spatial.KDTree(zip(x_s,y_s,z_s))
#     return tree, ids

# tree, ids = create_tree('../data/point_cloud_no_label.csv')
# print 'done'

In [8]:
# # Get Nearest Neighbors (grain_id, x, y, z)
# def get_node_coors_from_id(node_id, dbpath):
#     conn = sqlite3.connect(dbpath)
#     cursor = conn.execute("SELECT x, y, z  from crack_nodes where node_id = " + str(node_id))
#     for row in cursor:
#         x = row[0]
#         y =  row[1]
#         z = row[2]
#     conn.close()
#     return x, y, z

In [9]:
# def nearest_neighbor_ids_coors(tree, x, y, z, radius, ids):
#     neighbors = tree.query_ball_point((x,y,z), radius)
#     grain_ids_coors = []
#     for i in neighbors:
#         grain_ids_coors.append((ids[int((tree.data[i][0]/6)+(67*(tree.data[i][1]/6))+(8375*(tree.data[i][2])/6))], 
#                           int(tree.data[i][0]), int(tree.data[i][1]), int(tree.data[i][2])))

#     return grain_ids_coors

In [10]:
def alter_tables_grain_weight(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        conn.execute('alter table neighbor_%d' % r + 
                    ' add column num_xtal int')
        conn.execute('alter table neighbor_%d' % r + 
                    ' add column rank int')
        conn.execute('alter table neighbor_%d' % r + 
                    ' add column weight float')
    conn.close()

In [11]:
# Nearest Neighbors - Grain Weight Features
def nn_features(dbpath):
    import sqlite3
    import operator
    num_cracks = 9
    
    conn = sqlite3.connect(dbpath)
    
    for r in range(6,43,2):#(6,61,3):
        x = 0
        for crack_id in range(1,num_cracks + 1):
            cursor = conn.execute('select distinct arr_index from neighbor_%d where crack_id=%d' % (r, crack_id))
            for row in cursor:
                neighbors = []
                cursor2 = conn.execute('select grain_id from neighbor_%d where crack_id=%d and arr_index=%d' \
                                      % (r, crack_id, row[0]))
                for row2 in cursor2:
                    grain_id_ = row2[0]
                    grain_id = grain_id_ % 10000
                    neighbors.append(grain_id)

                pts_grain = {}
                for i in neighbors:
                    if i in pts_grain:
                        pts_grain[i] += 1
                    else:
                        pts_grain[i] = 1

                sorted_pts = sorted(pts_grain.items(), key=operator.itemgetter(1), reverse=True)
                total_pts = len(neighbors)
                rank = 1
                for i in range(len(sorted_pts)):
                    num_pts = sorted_pts[i][1]
                    if num_pts < sorted_pts[i - 1][1] and i > 0:
                        rank += 1
                    weight = float(num_pts) / float(total_pts)
                    num_xtal = len(sorted_pts)
                    conn.execute("update neighbor_%d set num_xtal=%d, rank=%d, weight=%f "%(r,num_xtal,rank,weight)+
                                 "where crack_id=%d and arr_index=%d and grain_id %% 10000 = %d" \
                                 % (crack_id, row[0], sorted_pts[i][0]))
        print r
    conn.commit()
    conn.close()

In [12]:
def create_data_dbs(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        conn.execute('create table data_%d'%r+' (crack_id int,arr_index int,theta float,x float,y float,z float,' +
                     'num_xtal int, primary key(crack_id, arr_index))')
        
        for i in range(1,11):
            conn.execute('alter table data_%d'%r+' add column grain_id_%d int'%i)
            conn.execute('alter table data_%d'%r+' add column phi1_%d int'%i)
            conn.execute('alter table data_%d'%r+' add column Phi_%d int'%i)
            conn.execute('alter table data_%d'%r+' add column phi2_%d int'%i)
            conn.execute('alter table data_%d'%r+' add column weight_%d float'%i)

        for i in range(1,11):
            for j in range(i + 1, 11):
                conn.execute('alter table data_%d'%r+' add column mis_%dto%d float'%(i,j))
    conn.close()
    print 'done'

In [13]:
import sqlite3

def setup_data_dbs(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        conn.execute('insert into data_%d'%r + ' (crack_id,arr_index) select crack_id,arr_index from crack_nodes')
        conn.commit()
        cursor = conn.execute('select crack_id,arr_index from data_%d'%r)
        for row in cursor:
            cursor2 = conn.execute('select theta,x,y,z from crack_nodes where crack_id=%d and arr_index=%d'%(row[0],row[1]))
            for row2 in cursor2:
                conn.execute('update data_%d set theta=%f,x=%f,y=%f,z=%f where \
                             crack_id=%d and arr_index=%d'%(r,row2[0],row2[1],row2[2],row2[3],row[0],row[1]))
    conn.commit()
    
    conn.close()
    print 'done'

In [14]:
import sqlite3

def save_phis(dbpath, db2path):

    conn = sqlite3.connect(dbpath)
    grain_id_short = -1 # These are here just cause it makes it work...
    arr_index = -1
    for r in range(6,43,2):#(6,61,3):
        cursor = conn.execute('select distinct crack_id, arr_index, (grain_id % 10000)as gid, num_xtal, rank, weight ' +
                              'from neighbor_%d order by crack_id asc, arr_index asc, rank asc'%r)
        x = 0
        num = 1
        for row in cursor:
            crack_id=row[0]
            if grain_id_short != row[2]:
                if x > 0:
                    num += 1
            if arr_index != row[1]:
                num = 1
            arr_index=row[1]
            grain_id_short=row[2]
            num_xtal=row[3]
            rank=row[4]
            weight=row[5]
            
            conn2 = sqlite3.connect(db2path)
            cursor2 = conn2.execute('select distinct phi1,Phi,phi2 from euler_angles where grain_id%%10000=%d'%grain_id_short)
            for row2 in cursor2:
                phi1=row2[0]
                Phi=row2[1]
                phi2=row2[2]

            if (num > 0 and num <= 10):
                conn.execute('update data_%d set grain_id_%d=%d where crack_id=%d and arr_index=%d' \
                            % (r, num, grain_id_short, crack_id, arr_index))
                conn.execute('update data_%d set phi1_%d=%f,Phi_%d=%f,phi2_%d=%f where crack_id=%d and arr_index=%d' \
                            % (r, num, phi1, num, Phi, num, phi2, crack_id, arr_index))
                conn.execute('update data_%d set num_xtal=%d,weight_%d=%f where crack_id=%d and arr_index=%d' \
                            % (r, num_xtal, num, weight, crack_id, arr_index))
            x+=1
        print r
            
    conn.commit()
    conn.close()
    print 'done'

In [15]:
# Calculate misorientation between two grains with Euler Angles

from numpy import *

#######################################################
def get_R_o(EAngles):

    # returns rotation matrix
            
    R_o = zeros((3,3),dtype=double)
    
    euler_phi1 = EAngles[0]
    euler_Phi = EAngles[1]
    euler_phi2 = EAngles[2]
    
    R_o[0,0]= cos(euler_phi1)*cos(euler_phi2) - sin(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,0]= -cos(euler_phi1)*sin(euler_phi2) - sin(euler_phi1)*cos(euler_phi2)*cos(euler_Phi)
    R_o[2,0]= sin(euler_phi1)*sin(euler_Phi)
    R_o[0,1]= sin(euler_phi1)*cos(euler_phi2) + cos(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,1]= -sin(euler_phi1)*sin(euler_phi2) + cos(euler_phi1)*cos(euler_phi2)*cos(euler_Phi) 
    R_o[2,1]= -cos(euler_phi1)*sin(euler_Phi)
    R_o[0,2]= sin(euler_phi2)*sin(euler_Phi)  
    R_o[1,2]= cos(euler_phi2)*sin(euler_Phi)
    R_o[2,2]= cos(euler_Phi) 

    return asmatrix(R_o)

#######################################################
def symmetric_operators():

    # returns 24 cubic symmetric operators

    sym_op = []

    sym_op.append(asmatrix([[1,0,0], [0,1,0], [0,0,1]]))
    sym_op.append(asmatrix([[0,0,1], [1,0,0], [0,1,0]]))
    sym_op.append(asmatrix([[0,1,0], [0,0,1], [1,0,0]]))
    sym_op.append(asmatrix([[0,-1,0], [0,0,1], [-1,0,0]])) 
    sym_op.append(asmatrix([[0,-1,0], [0,0,-1], [1,0,0]]))
    sym_op.append(asmatrix([[0,1,0], [0,0,-1], [-1,0,0]]))
    sym_op.append(asmatrix([[0,0,-1], [1,0,0], [0,-1,0]]))
    sym_op.append(asmatrix([[0,0,-1], [-1,0,0], [0,1,0]]))
    sym_op.append(asmatrix([[0,0,1], [-1,0,0], [0,-1,0]]))
    sym_op.append(asmatrix([[-1,0,0], [0,1,0], [0,0,-1]]))
    sym_op.append(asmatrix([[-1,0,0], [0,-1,0], [0,0,1]]))
    sym_op.append(asmatrix([[1,0,0], [0,-1,0], [0,0,-1]]))
    sym_op.append(asmatrix([[0,0,-1], [0,-1,0], [-1,0,0]])) 
    sym_op.append(asmatrix([[0,0,1], [0,-1,0], [1,0,0]]))
    sym_op.append(asmatrix([[0,0,1], [0,1,0], [-1,0,0]]))
    sym_op.append(asmatrix([[0,0,-1], [0,1,0], [1,0,0]]))
    sym_op.append(asmatrix([[-1,0,0], [0,0,-1], [0,-1,0]]))
    sym_op.append(asmatrix([[1,0,0], [0,0,-1], [0,1,0]]))
    sym_op.append(asmatrix([[1,0,0], [0,0,1], [0,-1,0]]))
    sym_op.append(asmatrix([[-1,0,0], [0,0,1], [0,1,0]]))
    sym_op.append(asmatrix([[0,-1,0], [-1,0,0], [0,0,-1]]))
    sym_op.append(asmatrix([[0,1,0], [-1,0,0], [0,0,1]]))
    sym_op.append(asmatrix([[0,1,0], [1,0,0], [0,0,-1]]))
    sym_op.append(asmatrix([[0,-1,0], [1,0,0], [0,0,1]]))

    return sym_op

#######################################################
def get_disorientation_1(phi1_1, Phi_1, phi2_1, phi1_2, Phi_2, phi2_2):

    # uses quaternions

    # these should be in radians
    phi1 = [phi1_1, phi1_2]
    phi = [Phi_1, Phi_2]
    phi2 = [phi2_1, phi2_2]

    Q = {}
    for i in range(2):
        s = sin(0.5 * phi[i])
        c = cos(0.5 * phi[i])
        s1 = sin(0.5 * (phi1[i]-phi2[i]))
        c1 = cos(0.5 * (phi1[i]-phi2[i]))
        s2 = sin(0.5 * (phi1[i]+phi2[i]))
        c2 = cos(0.5 * (phi1[i]+phi2[i]))

        q1 = s*c1
        q2 = s*s1
        q3 = c*s2
        q4 = c*c2

        Q[i] = [q1, q2, q3, q4]

    qresult = [0, 0, 0, 0]
    qresult[0] = abs(Q[0][0]*Q[1][3]-Q[0][3]*Q[1][0]+Q[0][1]*Q[1][2]-Q[0][2]*Q[1][1])
    qresult[1] = abs(Q[0][1]*Q[1][3]-Q[0][3]*Q[1][1]+Q[0][2]*Q[1][0]-Q[0][0]*Q[1][2])
    qresult[2] = abs(Q[0][2]*Q[1][3]-Q[0][3]*Q[1][2]+Q[0][0]*Q[1][1]-Q[0][1]*Q[1][0])
    qresult[3] = abs(Q[0][3]*Q[1][3]+Q[0][0]*Q[1][0]+Q[0][1]*Q[1][1]+Q[0][2]*Q[1][2])

    temp = sort(qresult)

    maximum = temp[3]
    maximum2 = temp[2]

    disor = maximum
    if maximum < (maximum+maximum2)/sqrt(2.0):
        disor = (maximum+maximum2)/sqrt(2.0)

    rtmp = sum(qresult)/2.0
    if disor < rtmp:
        disor = rtmp

    if disor > 1:
        disor = 1.0

    if disor < -1:
        disor = -1.0

    return arccos(disor)*360/pi

#######################################################
def get_disorientation_2(phi1_1, Phi_1, phi2_1, phi1_2, Phi_2, phi2_2):

    # uses 24 cubic symmetric operators

    # get Euler angles (radians)
    mat_1_angles = [phi1_1, Phi_1, phi2_1]
    mat_2_angles = phi1_2, Phi_2, phi2_2

    # calculate orientation matrix of grain 1
    g_A = get_R_o([mat_1_angles[0],mat_1_angles[1],mat_1_angles[2]])

    # calculate orientation matrix of grain 2
    g_B = get_R_o([mat_2_angles[0],mat_2_angles[1],mat_2_angles[2]])

    # misorientation matrix
    del_g = g_B * linalg.inv(g_A)

    # get symmetric operators
    sym_op = symmetric_operators()

    # containers for misorientation angles and rotation axes
    misorientations = []
    rotation_axes = {}

    # calculate misorientations
    for i in range(24):
        r = sym_op[i] * del_g
        misorientations.append(rad2deg(arccos((trace(r)-1)/2.0)))

    # return disorientation
    return min(misorientations)

In [16]:
import sqlite3

def save_misor_data(dbpath):

    conn = sqlite3.connect(dbpath)
    
    for r in range(6,43,2):#(6,61,3):
        cursor = conn.execute('select crack_id, arr_index, num_xtal, ' +
            'phi1_1, phi1_2, phi1_3, phi1_4, phi1_5, phi1_6, phi1_7, phi1_8, phi1_9, phi1_10, ' +
            'Phi_1, Phi_2, Phi_3, Phi_4, Phi_5, Phi_6, Phi_7, Phi_8, Phi_9, Phi_10, ' +
            'phi2_1, phi2_2, phi2_3, phi2_4, phi2_5, phi2_6, phi2_7, phi2_8, phi2_9, phi2_10 from data_%d'%r)
        x=0
        for row in cursor:
            crack_id=row[0]
            arr_index=row[1]
            num_xtal=row[2]
            
            phi1s = []
            Phis = []
            phi2s = []
            
            for i in range(3,13):
                phi1s.append(row[i])
                Phis.append(row[i+10])
                phi2s.append(row[i+20])
            
            if num_xtal > 1:
                if num_xtal > 10: # We don't care about more than 10 grains
                    num_xtal = 10
                for i in range(num_xtal):
                    for j in range(i + 1, num_xtal):
                        mis = get_disorientation_1(phi1s[i],Phis[i],phi2s[i],phi1s[j],Phis[j],phi2s[j])
                        conn.execute('update data_%d set mis_%dto%d=%f where crack_id=%d and arr_index=%d' \
                                 %(r,i+1,j+1,mis,crack_id,arr_index))
            x+=1
        print r
    conn.commit()
    conn.close()
    print 'done'

In [17]:
def remove_pointless_data(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):#(6,61,3):
        x = 0
        cursor = conn.execute('select crack_id,arr_index,num_xtal from data_%d'%r)
        for row in cursor:
            if row[2] == None:
                conn.execute("delete from data_%d where crack_id=%d and arr_index=%d"%(r,row[0],row[1]))
    conn.commit()
    conn.close()
    print 'done'

In [18]:
alter_tables_grain_weight('../../big-data-smaller-r/base.db')
print 'done'

done


In [19]:
nn_features('../../big-data-smaller-r/base.db')
print 'done'

6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
done


In [20]:
create_data_dbs('../../big-data-smaller-r/base.db')

done


In [21]:
setup_data_dbs('../../big-data-smaller-r/base.db')

done


In [22]:
save_phis('../../big-data-smaller-r/base.db', '../../big-data-smaller-r/near.db')

6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
done


In [23]:
save_misor_data('../../big-data-smaller-r/base.db')

6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
done


In [24]:
remove_pointless_data('../../big-data-smaller-r/base.db')

done
